In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df_od = pd.read_csv("orders_eda.csv")
df_opp = pd.read_csv("order_products_prior_eda.csv")
df_opt = pd.read_csv("order_products_train_eda.csv")


In [3]:
df_od_opp = (
    df_od
    .merge(df_opp, on="order_id", how="left")
)

len(df_od_opp) == len(df_od)


False

In [5]:
df_od_opp.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0
1,2539329,1,prior,1,2,8,NaN,14084.0,2.0,0.0
2,2539329,1,prior,1,2,8,NaN,12427.0,3.0,0.0
3,2539329,1,prior,1,2,8,NaN,26088.0,4.0,0.0
4,2539329,1,prior,1,2,8,NaN,26405.0,5.0,0.0


In [6]:
df_od_opp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32640698 entries, 0 to 32640697
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
 7   product_id              float64
 8   add_to_cart_order       float64
 9   reordered               float64
dtypes: float64(4), int64(5), object(1)
memory usage: 2.4+ GB


In [4]:
df_prior = df_od_opp[df_od_opp["eval_set"] == "prior"]


In [8]:
user_total_orders = (
    df_prior
    .groupby("user_id")["order_number"]
    .max()
    .reset_index(name="user_total_orders")
)


In [9]:
user_total_products = (
    df_prior
    .groupby("user_id")
    .size()
    .reset_index(name="user_total_products")
)


In [10]:
basket_size = (
    df_prior
    .groupby(["user_id", "order_id"])
    .size()
    .reset_index(name="basket_size")
)

user_avg_basket = (
    basket_size
    .groupby("user_id")["basket_size"]
    .mean()
    .reset_index(name="user_avg_basket_size")
)


In [11]:
user_reorder_ratio = (
    df_prior
    .groupby("user_id")["reordered"]
    .mean()
    .reset_index(name="user_reorder_ratio")
)


In [12]:
user_avg_days_between = (
    df_prior
    .groupby("user_id")["days_since_prior_order"]
    .mean()
    .reset_index(name="user_avg_days_between_orders")
)


In [13]:
user_features = (
    user_total_orders
    .merge(user_total_products, on="user_id", how="left")
    .merge(user_avg_basket, on="user_id", how="left")
    .merge(user_reorder_ratio, on="user_id", how="left")
    .merge(user_avg_days_between, on="user_id", how="left")
)


In [14]:
user_features.head()

,user_id,user_total_orders,user_total_products,user_avg_basket_size,user_reorder_ratio,user_avg_days_between_orders
0,1,10,59,5.900000,0.694915,20.259259
1,2,14,195,13.928571,0.476923,15.967033
2,3,12,88,7.333333,0.625000,11.487179
3,4,5,18,3.600000,0.055556,15.357143
4,5,4,37,9.250000,0.378378,14.500000


In [15]:
# user_features.to_csv("user_features.csv", index=False)

In [16]:
prod_total_purchases = (
    df_prior
    .groupby("product_id")
    .size()
    .reset_index(name="prod_total_purchases")
)


In [17]:
prod_unique_users = (
    df_prior
    .groupby("product_id")["user_id"]
    .nunique()
    .reset_index(name="prod_unique_users")
)


In [18]:
prod_reorder_rate = (
    df_prior
    .groupby("product_id")["reordered"]
    .mean()
    .reset_index(name="prod_reorder_rate")
)


In [19]:
prod_order_count = (
    df_prior
    .groupby("product_id")["order_id"]
    .nunique()
    .reset_index(name="prod_order_count")
)


In [20]:
prod_avg_cart_pos = (
    df_prior
    .groupby("product_id")["add_to_cart_order"]
    .mean()
    .reset_index(name="prod_avg_cart_position")
)


In [21]:
product_features = (
    prod_total_purchases
    .merge(prod_unique_users, on="product_id", how="left")
    .merge(prod_reorder_rate, on="product_id", how="left")
    .merge(prod_avg_cart_pos, on="product_id", how="left")
    .merge(prod_order_count, on="product_id", how="left")
)


In [22]:
product_features.shape
product_features["product_id"].is_unique
product_features.head()


,product_id,prod_total_purchases,prod_unique_users,prod_reorder_rate,prod_avg_cart_position,prod_order_count
0,1.0,1852,716,0.613391,5.801836,1852
1,2.0,90,78,0.133333,9.888889,90
2,3.0,277,74,0.732852,6.415162,277
3,4.0,329,182,0.446809,9.507599,329
4,5.0,15,6,0.600000,6.466667,15


In [23]:
# product_features.to_csv("product_features.csv", index=False)


In [24]:
up_times_bought = (
    df_prior
    .groupby(["user_id", "product_id"])
    .size()
    .reset_index(name="up_times_bought")
)


In [25]:
up_first_order = (
    df_prior
    .groupby(["user_id", "product_id"])["order_number"]
    .min()
    .reset_index(name="up_first_order_number")
)


In [26]:
up_last_order = (
    df_prior
    .groupby(["user_id", "product_id"])["order_number"]
    .max()
    .reset_index(name="up_last_order_number")
)


In [27]:
user_total_orders = (
    df_prior
    .groupby("user_id")["order_number"]
    .max()
    .reset_index(name="user_total_orders")
)

up_order_rate = (
    up_times_bought
    .merge(user_total_orders, on="user_id", how="left")
)

up_order_rate["up_order_rate"] = (
    up_order_rate["up_times_bought"] /
    up_order_rate["user_total_orders"]
)

up_order_rate = up_order_rate[
    ["user_id", "product_id", "up_order_rate"]
]


In [28]:
up_reorder_ratio = (
    df_prior
    .groupby(["user_id", "product_id"])["reordered"]
    .mean()
    .reset_index(name="up_reorder_ratio")
)


In [29]:
up_avg_cart_position = (
    df_prior
    .groupby(["user_id", "product_id"])["add_to_cart_order"]
    .mean()
    .reset_index(name="up_avg_cart_position")
)


In [30]:
up_orders_since_last = (
    up_last_order
    .merge(user_total_orders, on="user_id", how="left")
)

up_orders_since_last["up_orders_since_last_bought"] = (
    up_orders_since_last["user_total_orders"] -
    up_orders_since_last["up_last_order_number"]
)

up_orders_since_last = up_orders_since_last[
    ["user_id", "product_id", "up_orders_since_last_bought"]
]


In [31]:
user_product_features = (
    up_times_bought
    .merge(up_first_order, on=["user_id", "product_id"])
    .merge(up_last_order, on=["user_id", "product_id"])
    .merge(up_order_rate, on=["user_id", "product_id"])
    .merge(up_reorder_ratio, on=["user_id", "product_id"])
    .merge(up_avg_cart_position, on=["user_id", "product_id"])
    .merge(up_orders_since_last, on=["user_id", "product_id"])
)


In [34]:
user_product_features.head()
user_product_features.shape




(13307953, 9)

In [35]:
user_product_features.to_csv("user_product_features.csv", index=False)


In [5]:
user_product_features = pd.read_csv("user_product_features.csv")
product_features = pd.read_csv("product_features.csv")
user_features= pd.read_csv("user_features.csv")

In [11]:
candidates = df_prior[["user_id", "product_id"]]


NameError: name 'df_prior' is not defined

In [7]:
candidates = candidates.drop_duplicates()


In [8]:
candidates.head()


,user_id,product_id
0,1,196.0
1,1,14084.0
2,1,12427.0
3,1,26088.0
4,1,26405.0


In [9]:
candidates.duplicated(
    subset=["user_id", "product_id"]
).any()


np.False_

In [10]:
candidates[candidates["user_id"] == 1]


,user_id,product_id
0,1,196.0
1,1,14084.0
2,1,12427.0
3,1,26088.0
4,1,26405.0
6,1,10258.0
8,1,13176.0
10,1,13032.0
14,1,25133.0
15,1,30450.0


In [11]:
main_training_data = (
    candidates
    .merge(user_features, on="user_id", how="left")
    .merge(product_features, on="product_id", how="left")
    .merge(
        user_product_features,
        on=["user_id", "product_id"],
        how="left"
    )
)


In [7]:
main_training_data.head()

NameError: name 'main_training_data' is not defined

In [13]:
main_training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 19 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   user_id                       int64  
 1   product_id                    float64
 2   user_total_orders             int64  
 3   user_total_products           int64  
 4   user_avg_basket_size          float64
 5   user_reorder_ratio            float64
 6   user_avg_days_between_orders  float64
 7   prod_total_purchases          int64  
 8   prod_unique_users             int64  
 9   prod_reorder_rate             float64
 10  prod_avg_cart_position        float64
 11  prod_order_count              int64  
 12  up_times_bought               int64  
 13  up_first_order_number         int64  
 14  up_last_order_number          int64  
 15  up_order_rate                 float64
 16  up_reorder_ratio              float64
 17  up_avg_cart_position          float64
 18  up_orders_since_last

In [14]:
len(main_training_data) == len(candidates)


True

In [15]:
df_opt.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [16]:
train_orders = df_od_opp[df_od_opp["eval_set"] == "train"]

train_truth = (
    train_orders[["order_id", "user_id"]]
    .merge(df_opt, on="order_id", how="inner")
)

In [18]:
train_truth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 5 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   user_id            1384617 non-null  int64
 2   product_id         1384617 non-null  int64
 3   add_to_cart_order  1384617 non-null  int64
 4   reordered          1384617 non-null  int64
dtypes: int64(5)
memory usage: 52.8 MB


In [19]:
train_truth = train_truth[
    ["user_id", "product_id"]
].drop_duplicates()

train_truth["target"] = 1


In [ ]:
train_truth.to_csv("train_truth.csv", index=False)


In [6]:
train_truth=pd.read_csv("train_truth.csv")
main_training_data=pd.read_csv("main_training_data.csv")

MemoryError: Unable to allocate 914. MiB for an array with shape (9, 13307953) and data type float64

In [5]:
main_training_data = (
    main_training_data
    .merge(
        train_truth,
        on=["user_id", "product_id"],
        how="left"
    )
)


In [6]:
main_training_data["target"] = (
    main_training_data["target"]
    .fillna(0)
    .astype(int)
)


In [7]:
main_training_data = main_training_data.drop(
    columns=["order_id", "eval_set"],
    errors="ignore"
)


In [8]:
main_training_data.head()

,user_id,product_id,user_total_orders,user_total_products,user_avg_basket_size,user_reorder_ratio,user_avg_days_between_orders,prod_total_purchases,prod_unique_users,prod_reorder_rate,prod_avg_cart_position,prod_order_count,up_times_bought,up_first_order_number,up_last_order_number,up_order_rate,up_reorder_ratio,up_avg_cart_position,up_orders_since_last_bought,target
0,1,196.0,10,59,5.9,0.694915,20.259259,35791,8000,0.776480,3.721774,35791,10,1,10,1.0,0.9,1.4,0,1
1,1,14084.0,10,59,5.9,0.694915,20.259259,15935,3012,0.810982,5.792595,15935,1,1,1,0.1,0.0,2.0,9,0
2,1,12427.0,10,59,5.9,0.694915,20.259259,6476,1679,0.740735,4.760037,6476,10,1,10,1.0,0.9,3.3,0,0
3,1,26088.0,10,59,5.9,0.694915,20.259259,2523,1163,0.539041,6.495838,2523,2,1,2,0.2,0.5,4.5,8,1
4,1,26405.0,10,59,5.9,0.694915,20.259259,1214,678,0.441516,3.116969,1214,2,1,4,0.2,0.5,5.0,6,1


In [ ]:
main_training_data["target"].value_counts(normalize=True)


target
0    0.93772
1    0.06228
Name: proportion, dtype: float64

In [13]:
main_training_data.duplicated(
    subset=["user_id", "product_id"]
).any()


np.False_

In [14]:
main_training_data.to_csv("main_training_data.csv", index=False)